In [ ]:
# Warning!!!: On all those variables, don't remove the starting/ending quotes when doing copy/paste
# Enter the main namespace name of this lab. It should be in the form xraylab-{guid}, like xraylab-f2d5
namespace = 'xraylab-0001'

# Enter you bucket base name. That's the same one you have put in the config map. It should be identical as your namespace if you have followed instructions.
bucket_base_name = 'xraylab-0001'

# Enter you Access and Secret keys. They're the ones that were displayed in the instructions.
aws_access_key_id = '02JMAK69UEON1JGYKSN9'
aws_secret_access_key = 'fedp7q3LodO2hiGQyP2SVJSTDwL6BfScMjBLs0ey'

# Do not change this value, this is the internal location for the RGW
endpoint_url = 'http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local'

attributes = {}
attributes['push-endpoint'] = 'kafka://my-cluster-kafka-bootstrap.'+namespace+':9092'
attributes['kafka-ack-level'] = 'broker'

In [ ]:
pip install boto3

In [ ]:
import boto3
import json
import botocore
import argparse

In [ ]:
s3 = boto3.client('s3',
                endpoint_url = endpoint_url,
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

sns = boto3.client('sns', 
                endpoint_url = endpoint_url, 
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key= aws_secret_access_key,
                region_name='default', 
                config=botocore.client.Config(signature_version = 's3'))

In [ ]:
for bucket in s3.list_buckets()['Buckets']:
    name = bucket['Name']
    print("BUCKET: " + name)

In [ ]:
bucket = s3.list_objects_v2(Bucket='xraylab-0001-anonymized')
for obj in bucket['Contents']:
    print(obj['Key'])

In [ ]:
for bucket in s3.list_buckets()['Buckets']:
    bucket_policy = {
                      "Version":"2012-10-17",
                      "Statement":[
                        {
                          "Sid":"AddPerm",
                          "Effect":"Allow",
                          "Principal": "*",
                          "Action":["s3:GetObject"],
                          "Resource":["arn:aws:s3:::{0}/*".format(bucket['Name'])]
                        }
                      ]
                    }
    bucket_policy = json.dumps(bucket_policy)
    s3.put_bucket_policy(Bucket=bucket['Name'], Policy=bucket_policy)

In [ ]:
def create_topic(topic):
    topic_arn = sns.create_topic(Name=topic, Attributes=attributes)['TopicArn']
    return topic_arn

In [ ]:
create_topic('xray-images-anonymized')

In [ ]:
sns.list_topics()

In [ ]:
bucket_notifications_configuration = {
            "TopicConfigurations": [
                {
                    "Id": 'xray-images-anonymized',
                    "TopicArn": 'arn:aws:sns:s3a::xray-images-anonymized',
                    "Events": ["s3:ObjectCreated:*"]
                }
            ]
        }

bucket_name = 'xraylab-0001-anonymized'
s3.put_bucket_notification_configuration(Bucket = bucket_name,
        NotificationConfiguration=bucket_notifications_configuration)